# 1.7 第一个应用：鸢尾花(Iris)分类

## 1.7.1 初识数据

In [ ]:
# 调用load_iris函数来加载数据
from sklearn.datasets import load_iris
iris_dataset = load_iris()

In [2]:
# load_iris返回的iris对象是一个Bunch对象，与字典类似，里面包含键和值
print("Keys of iris_dataset: \n{}".format(iris_dataset.keys()))

Keys of iris_dataset: 
dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [12]:
# DESCR键对应的值是数据集的简要说明。下面给出说明的开头部分
print(iris_dataset['DESCR'][:440] + "\n...")

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica


...


In [14]:
# target_names键对应的值是一个字符串数组，里面包含的是我们要预测的花的品种
print("Target names: \n{}".format(iris_dataset['target_names']))

Target names: 
['setosa' 'versicolor' 'virginica']


In [ ]:
# feature_names键对应的值是一个字符串列表，对每一个特征进行了说明: sepal - 花萼， petal - 花瓣
print("Feature names: \n{}".format(iris_dataset['feature_names']))

Feature names: 
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [16]:
# 数据包含在target和data字段中。
# data里面是花萼长度、花萼宽带、花瓣长度、花瓣宽带的测量数据，格式是NumPy数组
print("Type of data: {}".format(type(iris_dataset['data'])))

Type of data: <class 'numpy.ndarray'>


In [18]:
# data数组的每一行对应一朵花，列代表每朵花的四个测量数据
print("Shape of data: {}".format(iris_dataset['data'].shape))

Shape of data: (150, 4)


In [19]:
# 上面数组中包含150朵不同的花的测量数据
# 机器学习的个体叫作样本(sample)，其属性叫作特征(feature)，data数组的形状(shape)是样本数乘以特征数。
# 下面是前5个样本的特征数值
print("First 5 rows of data:\n{}".format(iris_dataset['data'][:5]))

First 5 rows of data:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
